In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import to_categorical 

In [32]:
# Generate a toy dataset for classification
samples = 100
class_0 = np.random.multivariate_normal([-.5, -.5], .5 * np.eye(2), samples)
class_1 = np.random.multivariate_normal([.5, .5], .5 * np.eye(2), samples)
x = np.vstack((class_0, class_1))
y = np.array([0] * 100 + [1] * 100)

In [53]:
y_cat = to_categorical(y)
# create model
# https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/
model = Sequential()
model.add(Dense(16, input_dim=2, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='softmax'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y_cat, epochs = 10, verbose = 1);

Epoch 1/10
7/7 [==============================] - 0s 914us/step - loss: 0.6438 - accuracy: 0.7600
Epoch 2/10
7/7 [==============================] - 0s 1ms/step - loss: 0.6293 - accuracy: 0.8150
Epoch 3/10
7/7 [==============================] - 0s 2ms/step - loss: 0.6156 - accuracy: 0.8350
Epoch 4/10
7/7 [==============================] - 0s 2ms/step - loss: 0.6037 - accuracy: 0.8450
Epoch 5/10
7/7 [==============================] - 0s 1ms/step - loss: 0.5923 - accuracy: 0.8500
Epoch 6/10
7/7 [==============================] - 0s 2ms/step - loss: 0.5809 - accuracy: 0.8500
Epoch 7/10
7/7 [==============================] - 0s 1ms/step - loss: 0.5688 - accuracy: 0.8450
Epoch 8/10
7/7 [==============================] - 0s 2ms/step - loss: 0.5568 - accuracy: 0.8500
Epoch 9/10
7/7 [==============================] - 0s 2ms/step - loss: 0.5442 - accuracy: 0.8500
Epoch 10/10
7/7 [==============================] - 0s 2ms/step - loss: 0.5311 - accuracy: 0.8450


In [ ]:
# Visualize our test data points
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.scatter(x[y == 1, 0], x[y == 1, 1], alpha=0.3, color='red', label='class 1')
ax.scatter(x[y == 0, 0], x[y == 0, 1], alpha=0.3, color='blue', label='class 0')
# Plotting decision regions - https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html
x_min, x_max = x[:, 0].min() - 1, x[:, 0].max() + 1
y_min, y_max = x[:, 1].min() - 1, x[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])[:,0]
Z = Z.reshape(xx.shape)
Z = Z >.5
ax.contourf(xx, yy, Z, alpha=0.4)
# labels
ax.set_xlabel('x_1')
ax.set_ylabel('x_2')
ax.set_title('toy classification data with test data points')
ax.legend(loc='best')
plt.show()

In [70]:
model.get_weights()

[array([[-0.07477671,  0.14629981,  0.29175043,  0.53102726, -0.31878936,
         -0.5769781 ,  0.03163872, -0.21571194, -0.4005302 ,  0.03144066,
         -0.32418177, -0.5260415 ,  0.02695856, -0.22489293, -0.42192733,
         -0.19186117],
        [-0.2464494 ,  0.21801558,  0.11059012, -0.00524846,  0.04305302,
          0.01117597,  0.40254942,  0.24508126, -0.6243641 ,  0.5971447 ,
         -0.52108526, -0.5727419 , -0.28565708, -0.492268  ,  0.12328506,
         -0.2477745 ]], dtype=float32),
 array([-0.05925097,  0.05211407,  0.06283553,  0.0629601 , -0.04596082,
         0.01427127,  0.0663356 , -0.03231831,  0.07206664,  0.04932896,
         0.03112092,  0.05501873,  0.00271449, -0.03612828, -0.01298867,
         0.05565234], dtype=float32),
 array([[ 0.409836  , -0.46776325, -0.5107345 , -0.34403914],
        [ 0.5377591 ,  0.05549474, -0.33604574,  0.31432694],
        [ 0.33235252, -0.43304425,  0.35824096, -0.09056291],
        [ 0.47123155,  0.07510846,  0.1870441 , -0

In [71]:
    model2 = Sequential()
    model2.add(Dense(16, input_dim=2, activation='relu'))
    model2.add(Dense(4, activation='relu'))
    model2.add(Dense(2, activation='softmax'))
    # Compile model
    model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model2.set_params(model.get_params())

AttributeError: 'Sequential' object has no attribute 'set_params'

In [86]:
# random subspace sampler
models = []
for i in range(10):
    model2 = Sequential()
    model2.add(Dense(16, input_dim=2, activation='relu'))
    model2.add(Dense(4, activation='relu'))
    model2.add(Dense(2, activation='softmax'))
    # Compile model
    model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    weights = [model.get_weights()[j] + np.random.normal(size = model.get_weights()[j].shape) for j in range(len(model.get_weights()))]
    model2.set_weights(weights)
    models.append(model2)

In [ ]:
for i in range(len(test_points)):
    pt = test_points[i]
    if i == 0:
        ax.scatter(pt[0], pt[1], alpha=1., s=50, color='black', label='test data')
    else:
        ax.scatter(pt[0], pt[1], alpha=1., s=50, color='black')
## plot the decision boundary
x1 = np.arange(-5,5,.05)
ax.plot(x1, (-toymodel.intercept_-toymodel.coef_[0,0]*x1)/toymodel.coef_[0,1], label='decision boundary')